# Questions

In [ ]:
# We want to know how many facilities exist - simply produce a total count.
SELECT COUNT(*) FROM cd.facilities

#Produce a count of the number of facilities that have a cost to guests of 10 or more.
SELECT COUNT(*) FROM cd.facilities
WHERE guestcost >= 10

#Produce a count of the number of recommendations each member has made. Order by member ID.
SELECT recommendedby, count(*)
FROM cd.members
WHERE recommendedby IS NOT NULL
GROUP BY recommendedby
ORDER BY recommendedby

#Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.
SELECT facid, SUM(slots) AS "Total Slots"
FROM cd.bookings
GROUP BY facid
ORDER BY facid

#Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.
SELECT facid, SUM(slots) AS "Total Slots"
FROM cd.bookings
WHERE starttime >= '2012-09-01'
AND starttime < '2012-10-01'
GROUP BY facid
ORDER BY SUM(slots)

#Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.
SELECT facid, EXTRACT(MONTH FROM starttime) AS month, SUM(slots) AS "Total Slots"
FROM cd.bookings
WHERE EXTRACT(YEAR FROM starttime)= '2012'
GROUP BY facid, month
ORDER BY facid, month

#Find the total number of members (including guests) who have made at least one booking.
SELECT COUNT(DISTINCT memid) FROM cd.bookings

#Produce a list of facilities with more than 1000 slots booked. Produce an output table consisting of facility id and slots, sorted by facility id.
SELECT facid, SUM(slots) AS "Total Slots"
FROM cd.bookings
GROUP BY facid
HAVING SUM(slots) > 1000
ORDER BY facid

#Produce a list of facilities along with their total revenue. The output table should consist of facility name and revenue, 
#sorted by revenue. Remember that there's a different cost for guests and members!
SELECT fcs.name AS name, 
SUM(slots*CASE WHEN memid=0 THEN fcs.guestcost
ELSE fcs.membercost END) AS revenue
FROM cd.bookings bks
INNER JOIN cd.facilities fcs
ON fcs.facid=bks.facid
GROUP BY fcs.name
ORDER BY revenue

# Produce a list of facilities with a total revenue less than 1000. Produce an output table consisting of facility name and revenue, 
#sorted by revenue. Remember that there's a different cost for guests and members!
SELEC name, revenue FROM (
	SELECT facs.name, SUM(CASE
				WHEN memid = 0 THEN slots * facs.guestcost
				ELSE slots * membercost
			END) AS revenue
		FROM cd.bookings bks
		INNER JOIN cd.facilities facs
			ON bks.facid = facs.facid
		GROUP BY facs.name
	) AS agg WHERE revenue < 1000
ORDER BY revenue;  

#Output the facility id that has the highest number of slots booked.
#simple method
SELECT facid, SUM(slots) AS "Total Slots"
FROM cd.bookings
GROUP BY facid
ORDER BY SUM(slots) DESC 
LIMIT 1

#second method
SELECT facid, total FROM(
  SELECT facid, SUM(slots) total, RANK() OVER(ORDER BY SUM(slots) DESC) rank
  FROM cd.bookings
  GROUP BY facid) as ranked
  WHERE rank=1        

# Produce a list of the total number of slots booked per facility per month in the year of 2012. In this version, include output 
#rows containing totals for all months per facility, and a total for all months for all facilities. The output table should consist 
#of facility id, month and slots, sorted by the id and month. When calculating the aggregated values for all months and all facids, return null values in the month and facid columns.
#simple method
SELECT facid, EXTRACT(MONTH FROM starttime) AS month, SUM(slots) AS slots
FROM cd.bookings
WHERE EXTRACT(YEAR FROM starttime) = '2012'
GROUP BY ROLLUP(facid, month)
ORDER BY facid, month

#second method
with bookings as (
	select facid, extract(month from starttime) as month, slots
	from cd.bookings
	where
		starttime >= '2012-01-01'
		and starttime < '2013-01-01'
)
select facid, month, sum(slots) from bookings group by facid, month
union all
select facid, null, sum(slots) from bookings group by facid
union all
select null, null, sum(slots) from bookings
order by facid, month;

#Produce a list of the total number of hours booked per facility, remembering that a slot lasts half an hour. The output table should 
#consist of the facility id, name, and hours booked, sorted by facility id. Try formatting the hours to two decimal places.
SELECT fcs.facid AS facid, fcs.name AS name,
TRIM(TO_CHAR(SUM(bks.slots)/2.0, '9999999999999999D99')) AS "Total Hours"
FROM cd.facilities fcs
JOIN cd.bookings bks
ON bks.facid=fcs.facid
GROUP BY fcs.facid, fcs.name
ORDER BY facid

#Produce a list of each member name, id, and their first booking after September 1st 2012. Order by member ID.
SELECT mems.surname, mems.firstname, mems.memid, min(bks.starttime) AS starttime
FROM cd.members mems
JOIN cd.bookings bks
ON bks.memid=mems.memid
WHERE bks.starttime >= '2012-09-01 08:00:00'
GROUP BY mems.surname, mems.firstname, mems.memid
ORDER BY mems.memid

#Produce a list of member names, with each row containing the total member count. Order by join date, and include guest members.
SELECT COUNT(*) OVER(), firstname, surname
FROM cd.members
ORDER BY joindate

#Produce a monotonically increasing numbered list of members (including guests), ordered by their date of joining. Remember that member 
#IDs are not guaranteed to be sequential.
SELECT ROW_NUMBER() OVER(ORDER BY joindate), firstname, surname
FROM cd.members
ORDER BY joindate




